# Import Packages and Read Data

This assumes there is a subdirectory in your current working directory called data that contains the Excel files.

In [1]:
import pandas as pd
from array import array
from collections import defaultdict


deferred_factors = pd.read_excel('data/deferred_factors.xlsx', index_col='Age', usecols=['Age', 'Combined_Factor'])
salary_increase_rate = pd.read_excel('data/salary_increase_rate.xlsx')
inforce_data = pd.read_excel('data/inforce_2022_p31.xlsx')

deferred_factors.columns = [col_name.lower() for col_name in deferred_factors.columns]
salary_increase_rate.columns = [col_name.lower() for col_name in salary_increase_rate.columns]
inforce_data.columns = [col_name.lower() for col_name in inforce_data.columns]


## Define Global Variables

### Increase Rates
`INCREASE_RATES` is an array of salary increase rates, where the index of the array is the years of service minus 1.

### Deferred Factors
`DEFERRED_FACTORS` is a dictionary of combined factors, mapping `age` -> `combined_factor`. For any age not contained in the dictionary, `combined_factor` is defaulted to 1.

### Accrued Benefit Factor
An assumption. Set to `0.0182`.

### PVAB Factor
An assumption. Set to `12.5`

In [2]:
INCREASE_RATES = array('f', salary_increase_rate['increase'].tolist())
DEFERRED_FACTORS = defaultdict(lambda: 1, pd.Series(deferred_factors['combined_factor']).to_dict())
ACCRUED_BENEFIT_FACTOR = 0.0182
PVAB_FACTOR = 12.5

## Calculate FAS

The below code block defines a function call `calculate_fas` which calculates the Five Year Average Salary of a state pension contributor. This will be used to calculate their Accrued Benefit.

In [3]:

def calculate_fas(yos: int, comp: float, years_ahead=0) -> float:
    """
    Calculates the Five Year Average Salary (FAS) for a pension contributor.
    If the contributor has < 5 years of experience, returns 0
    If the contributor has more than 9 years of experience, then calculates
    the 3-year average salary.

    :param yos: Years of Service with state
    :param comp: Current compensation
    :param years_ahead: How many years in the future to perform the FAS calculation
    :returns: FAS in $
    """
    for i in range(years_ahead):
        comp = comp * (1 + INCREASE_RATES[yos])
        yos += 1
    cum_sum = count = 0
    yos -= 1
    if yos < 4 or yos > 49:
        return 0
    elif yos < 10:
        while yos > 0 and count < 5:
            comp = comp / (1 + INCREASE_RATES[yos - 1])
            cum_sum += comp
            count += 1
            yos -= 1
    else:
        while count < 3:
            comp = comp / (1 + INCREASE_RATES[yos - 1])
            cum_sum += comp
            count += 1
            yos -= 1
    return cum_sum / count


### Example FAS Calculations

The function `calculate_fas` contains a parameter `years_ahead` which allows you to calculate a contributor's `FAS` years in advance. 

In the below example, we first calculate the `FAS` for a contributer with 19 years of experience (`yos`) and a current compensation of $100,000 (`comp`). The resulting value is stored in the `fas_example` variable.

Next, we calculate the contributor's prior year compensation (`prior_year_comp`) by leveraging the `INCREASE_RATES` array. We then calculate the `FAS` for this contributor at `yos = 18` and `comp = prior_year_comp`; however, we set `years_ahead = 1`. This should result in the same number obtained in `fas_example`.

In [4]:
yos = 19
comp = 100000
fas_example = round(calculate_fas(yos, comp), 2)
prior_year_comp = comp / (1 + INCREASE_RATES[yos - 1])
fas_example_next_year = round(calculate_fas(yos - 1, prior_year_comp, 1), 2)
assert fas_example == fas_example_next_year, f"{'${:,.2f}'.format(fas_example)} != {'${:,.2f}'.format(fas_example_next_year)}"
print(f"{'${:,.2f}'.format(fas_example)} == {'${:,.2f}'.format(fas_example_next_year)}")

$93,387.90 == $93,387.90


### Add FAS calculations DataFrame

Now we can add `fas` and `fas_next_year` columns to our dataframe.

In [5]:
inforce_data['fas'] = inforce_data.apply(lambda row: calculate_fas(int(row['yos']), row['comp']), axis=1)
inforce_data['fas_next_year'] = inforce_data.apply(lambda row: calculate_fas(int(row['yos']), row['comp'], years_ahead=1), axis=1)
inforce_data.head()

,age,count,comp,yos,fas,fas_next_year
0,65,368,79921,35,75355.149462,77615.803895
1,57,310,75555,35,71238.577065,73375.734329
2,52,751,74812,32,70538.024318,72654.165001
3,52,4437,74659,27,70393.765139,72505.578046
4,62,530,73281,35,69094.489655,71167.324298


## Accrued Benefit & PVAB calculations

We can now leverage the `fas` calculations to calculate the Accrued Benefit and PVAB.

In [6]:
inforce_data['accrued_benefit'] = inforce_data['yos'] * inforce_data['fas'] * ACCRUED_BENEFIT_FACTOR
inforce_data['pvab'] = inforce_data['accrued_benefit'] * inforce_data['age'].map(DEFERRED_FACTORS) * PVAB_FACTOR
inforce_data['accrued_benefit_next_year'] = (inforce_data['yos'] + 1) * inforce_data['fas'] * ACCRUED_BENEFIT_FACTOR
inforce_data['pvab_next_year'] = inforce_data['accrued_benefit_next_year'] * (inforce_data['age'] + 1).map(DEFERRED_FACTORS) * PVAB_FACTOR
inforce_data['pvab_increase'] = inforce_data['pvab_next_year'] - inforce_data['pvab']
inforce_data.head()

,age,count,comp,yos,fas,fas_next_year,accrued_benefit,pvab,accrued_benefit_next_year,pvab_next_year,pvab_increase
0,65,368,79921,35,75355.149462,77615.803895,48001.230207,600015.377591,49372.693927,617158.674094,17143.296503
1,57,310,75555,35,71238.577065,73375.734329,45378.973590,318218.422840,46675.515693,351451.128820,33232.705980
2,52,751,74812,32,70538.024318,72654.165001,41081.345363,202554.586616,42365.137406,224041.786539,21487.199923
3,52,4437,74659,27,70393.765139,72505.578046,34591.496189,170555.909237,35872.662715,189707.290823,19151.381586
4,62,530,73281,35,69094.489655,71167.324298,44013.189910,441780.546424,45270.709622,488691.891480,46911.345056
